In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("map").getOrCreate()

In [8]:
kill_0 = 's3://502pubg/clean/kill.parquet'
kill_spark = spark.read.parquet(kill_0)

In [9]:
kill_spark.printSchema()

root
 |-- killed_by: string (nullable = true)
 |-- killer_placement: double (nullable = true)
 |-- killer_position_x: integer (nullable = true)
 |-- killer_position_y: integer (nullable = true)
 |-- map: string (nullable = true)
 |-- time: integer (nullable = true)
 |-- victim_placement: double (nullable = true)
 |-- victim_position_x: integer (nullable = true)
 |-- victim_position_y: integer (nullable = true)
 |-- id: long (nullable = true)



In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.pyplot import imread
from scipy.ndimage.filters import gaussian_filter
import matplotlib.cm as cm
from matplotlib.colors import Normalize

In [10]:
kill = kill_spark.select("*").toPandas()

In [11]:
edf = kill.loc[kill['map'] == 'ERANGEL']
mdf = kill.loc[kill['map'] == 'MIRAMAR']

In [12]:
def killer_victim_df_maker(kill_0):
    #find the position of the killers and players who were killed on the map 
    df = edf
    victim_x_df = df.filter(regex = 'victim_position_x')
    victim_y_df = df.filter(regex = 'victim_position_y')
    killer_x_df = df.filter(regex = 'killer_position_x')
    killer_y_df = df.filter(regex = 'killer_position_y')
    #ravel()the matrix
    victim_x_s = pd.Series(victim_x_df.values.ravel('F'))
    victim_y_s = pd.Series(victim_y_df.values.ravel('F'))
    killer_x_s = pd.Series(killer_x_df.values.ravel('F'))
    killer_y_s = pd.Series(killer_y_df.values.ravel('F'))
    
    vdata = {'x':victim_x_s, 'y':victim_y_s}
    kdata = {'x':killer_x_s, 'y':killer_y_s}
    
    victim_df = pd.DataFrame(data = vdata).dropna(how = 'any')
    victim_df = victim_df[victim_df['x'] > 0]
    killer_df = pd.DataFrame(data = kdata).dropna(how = 'any')
    killer_df = killer_df[killer_df['x'] > 0]
    return killer_df, victim_df

In [13]:
ekdf,evdf = killer_victim_df_maker(edf)
mkdf,mvdf = killer_victim_df_maker(mdf)

In [15]:
print(ekdf.head(10))#the killers positions on the map "erangel"
print(evdf.head(10))#the players position who were killed on the map "erangel"
print(mkdf.head(10))#the killers positions on the map "miramar"
print(mvdf.head(10))#the players position who were killed on the map "miramar"
print(len(ekdf), len(evdf), len(mkdf), len(mvdf))

        x       y
0  526713  669857
1  462594  644537
2  360094  394891
3  335182  157544
4  205344  270663
5  469621  647893
6  302518  579473
7  624428  602356
8  450102  620974
9  281149  558808
         x       y
1   463091  643535
2   360259  396052
3   333915  157494
4   209850  270024
5   459994  646792
6   297147  579977
7   626641  614474
8   449996  621004
9   288445  559878
10  637573  461594
        x       y
0  526713  669857
1  462594  644537
2  360094  394891
3  335182  157544
4  205344  270663
5  469621  647893
6  302518  579473
7  624428  602356
8  450102  620974
9  281149  558808
         x       y
1   463091  643535
2   360259  396052
3   333915  157494
4   209850  270024
5   459994  646792
6   297147  579977
7   626641  614474
8   449996  621004
9   288445  559878
10  637573  461594
4035806 3961863 4035806 3961863


In [16]:
plot_data_ev = evdf[['x','y']].values
plot_data_ek = ekdf[['x','y']].values
plot_data_mv = mvdf[['x','y']].values
plot_data_mk = mkdf[['x','y']].values

In [17]:
plot_data_ev = plot_data_ev * 4040 /800000
plot_data_ek = plot_data_ek * 4040 /800000
plot_data_mv = plot_data_mv * 976 /800000
plot_data_mk = plot_data_mk * 976 /800000

In [18]:
print(plot_data_ev)

[[ 2338.60955 -2118.85737]
 [ 1819.30795  2000.0626 ]
 [ 1686.27075   795.3447 ]
 ...
 [ 2236.04405 -2223.73072]
 [ 1439.0985    671.52375]
 [ 1583.7709    958.91925]]


In [19]:
def heatmap(x, y, s, bins = 100):
    heatmap, xedges, yedges = np.histogram2d(x, y, bins = bins)
    heatmap = gaussian_filter(heatmap, sigma = s)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    return heatmap.T, extent